In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
gabung1 = pd.read_csv("test.csv")
gabung1

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [3]:
columns_with_number = [col for col in gabung1 if gabung1[col].dtype in ['int64', 'float64']]
columns_with_word = [col for col in gabung1 if gabung1[col].dtype in ['bool', 'object']]
df_kategorikal=gabung1[columns_with_word]
df_kategorikal
df=df_kategorikal
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,False,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,False,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,False,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,False,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,False,Brence Harperez
...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,False,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,False,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,False,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,False,Kitakan Conale


In [4]:
def hamming_distance(row1, row2, max_distance):
    distance = 0
    for col in row1.index:
        if row1[col] != row2[col]:
            distance += 1
        if distance > max_distance:
            break
    return distance

def calculate_distance_matrix(df):
    num_rows = len(df)
    distance_categorical = np.zeros((num_rows, num_rows))
    for i in tqdm(range(num_rows), desc="Calculating distance matrix"):
        for j in range(i+1, num_rows):
            distance = hamming_distance(df.iloc[i], df.iloc[j], max_distance=10)
            distance_categorical[i, j] = distance
            distance_categorical[j, i] = distance  # symmetry
    return distance_categorical

distance_categorical = calculate_distance_matrix(df)
print(distance_categorical)


Calculating distance matrix: 100%|██████████| 4277/4277 [07:52<00:00,  9.06it/s] 

[[0. 4. 5. ... 5. 6. 4.]
 [4. 0. 6. ... 6. 5. 5.]
 [5. 6. 0. ... 4. 5. 5.]
 ...
 [5. 6. 4. ... 0. 6. 5.]
 [6. 5. 5. ... 6. 0. 6.]
 [4. 5. 5. ... 5. 6. 0.]]


In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from tqdm import tqdm
# Menggunakan Min-Max Scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(gabung1[columns_with_number])

# Fungsi untuk menghitung Euclidean Distance dengan NaN dianggap sebagai 1
def euclidean_distance(x1, x2):
    diff = np.abs(x1 - x2)    
    diff[np.isnan(diff)] = 1    
    diff[np.isnan(x1) & np.isnan(x2)] = 1  # Jika keduanya NaN, jaraknya dianggap 0    
    distance = np.nansum(diff ** 2)    
    return distance


def calculate_distances(X):
    num_rows = X.shape[0]
    distances = np.zeros((num_rows, num_rows))
    
    for i in tqdm(range(num_rows), desc="Calculating distances"):
        for j in range(num_rows):
            if i != j:
                distances[i, j] = euclidean_distance(X[i], X[j])
    
    return distances
# Menghitung jarak antar baris setelah di-scaler
distances = calculate_distances(X_scaled)

# Konversi hasil jarak ke DataFrame
distances_continous = pd.DataFrame(distances, index=gabung1.index, columns=gabung1.index)

# Tampilkan hasil jarak antar baris dalam DataFrame
print("\nJarak Euclidean antar baris continous setelah di-scaler (dengan aturan NaN):")
print(distances_continous)

Calculating distances: 100%|██████████| 4277/4277 [03:59<00:00, 17.85it/s]


Jarak Euclidean antar baris continous setelah di-scaler (dengan aturan NaN):
          0         1         2         3         4         5         6     \
0     0.000000  0.030493  0.002564  0.089438  0.013717  0.007693  1.005768   
1     0.030493  0.000000  0.043311  0.145349  0.026263  0.046775  1.020879   
2     0.002564  0.043311  0.000000  0.077902  0.025253  0.005129  1.016023   
3     0.089438  0.145349  0.077902  0.000000  0.127830  0.049147  1.047080   
4     0.013717  0.026263  0.025253  0.127830  0.000000  0.025524  1.006025   
...        ...       ...       ...       ...       ...       ...       ...   
4272  0.007851  0.056290  0.001442  0.072614  0.037270  0.006571  1.027079   
4273  0.037221  0.106002  0.020557  0.055793  0.084272  0.021233  1.070708   
4274  1.000000  1.020238  1.000000  1.070050  1.005865  1.005129  2.000000   
4275  1.011796  1.031959  1.011796  1.024791  1.017662  1.003246  2.000551   
4276  0.041019  0.112531  0.023073  0.074056  0.090627  0.028202

In [6]:
distance_matrix=distances_continous+distance_categorical
print(distance_matrix)
distance_matrix =distance_matrix.map(np.sqrt)
distance_matrix.to_csv('distance_matrix.csv', index=False)
print("\n gabungan dan akar baris continous ")
print(distance_matrix)
distance_matrix=distance_matrix.to_numpy()

          0         1         2         3         4         5         6     \
0     0.000000  4.030493  5.002564  5.089438  4.013717  4.007693  6.005768   
1     4.030493  0.000000  6.043311  4.145349  3.026263  3.046775  7.020879   
2     5.002564  6.043311  0.000000  5.077902  6.025253  6.005129  4.016023   
3     5.089438  4.145349  5.077902  0.000000  4.127830  4.049147  6.047080   
4     4.013717  3.026263  6.025253  4.127830  0.000000  3.025524  7.006025   
...        ...       ...       ...       ...       ...       ...       ...   
4272  3.007851  4.056290  5.001442  5.072614  4.037270  4.006571  6.027079   
4273  4.037221  3.106002  6.020557  4.055793  3.084272  3.021233  7.070708   
4274  6.000000  7.020238  5.000000  7.070050  7.005865  7.005129  6.000000   
4275  7.011796  6.031959  6.011796  5.024791  6.017662  6.003246  7.000551   
4276  4.041019  5.112531  5.023073  6.074056  5.090627  5.028202  6.077552   

          7         8         9     ...      4267      4268    

In [7]:
df =gabung1

def knn_impute(df, distance_matrix, k=2):
    imputed_data = df.copy()
    col_name = df.columns[0]
    for i in range(len(df)):
        if np.isnan(df.iloc[i][col_name]):
            print(f"Imputing missing value at row {i}, column {col_name}...")
            distances = distance_matrix[i]
            neighbors = np.argsort(distances)[1:]  # exclude self

            neighbor_values = []
            for neighbor in neighbors:
                value = df.iloc[neighbor, 0]

                if not np.isnan(value):
                    neighbor_values.append((neighbor, value))
                if len(neighbor_values) == k:
                    break
            print(f"Using neighbors: {[neighbor for neighbor, _ in neighbor_values]}")
            if neighbor_values:
                imputed_value = np.mean([value for _, value in neighbor_values])
                print(f"Imputed value: {imputed_value}")
                imputed_data.iloc[i, 0] = imputed_value
            else:
                print("No non-missing neighbors, recursively calling the function...")
                imputed_data.iloc[neighbors, :] = knn_impute(imputed_data.iloc[neighbors, :], distance_matrix, k)
                imputed_data.iloc[i, 0] = np.nanmean(imputed_data.iloc[neighbors, 0])
                print(f"Imputed value: {imputed_data.iloc[i, 0]}")
            print()
    return imputed_data


def knn_impute_categorical(df, distance_matrix, k=3):
    imputed_data = df.copy()
    col_name = df.columns[0]
    for i in range(len(df)):
        if df.iloc[i][col_name] is np.nan:
            print(f"Imputing missing value at row {i}, column {col_name}...")
            
            neighbors = np.argsort(distance_matrix[i])[:k+1][1:]  # exclude self
            print(f"Neighbors: {neighbors}")
            # Get categorical values from neighbors
            neighbor_values = [df.iloc[neighbor, 0] for neighbor in neighbors]
            print("Neighbor values:")
            for neighbor, value in zip(neighbors, neighbor_values):
                print(f"  Neighbor {neighbor}: {value}")
            # Remove NaN values from neighbor_values
            non_nan_values = [(value, distance_matrix[i, neighbors[j]]) for j, value in enumerate(neighbor_values) if value is not np.nan]
            if non_nan_values:
                # Count the occurrences of each value
                value_counts = {}
                for value, _ in non_nan_values:
                    value_counts[value] = value_counts.get(value, 0) + 1
                # Find the mode
                max_count = max(value_counts.values())
                modes = [value for value, count in value_counts.items() if count == max_count]
                if len(modes) == 1:
                    # If there's only one mode, use it
                    imputed_value = modes[0]
                else:
                    # If there are multiple modes, use the one with the closest neighbor
                    closest_mode = min([(value, distance) for value, distance in non_nan_values if value in modes], key=lambda x: x[1])
                    imputed_value = closest_mode[0]
            else:
                # If all neighbors have NaN values, impute with the most frequent value in the column
                imputed_value = df[col_name].value_counts().index[0]
            print(f"Using neighbors: {neighbors}")
            print(f"Imputed value: {imputed_value}")
            print()
            imputed_data.iloc[i, 0] = imputed_value
    return imputed_data
imputed_df = df.copy()
for col in df.columns:
    if df[col].dtype.kind in ['i', 'f']:  # continuous column
        imputed_df[col] = knn_impute(imputed_df[[col]], distance_matrix, k=4)[col]
    else:  # categorical column
        imputed_df[col] = knn_impute_categorical(imputed_df[[col]], distance_matrix, k=4)[col]
print("Imputed dataframe:")
print(imputed_df)    

Imputing missing value at row 276, column HomePlanet...
Neighbors: [2010 4234 1824 1293]
Neighbor values:
  Neighbor 2010: Europa
  Neighbor 4234: Europa
  Neighbor 1824: Europa
  Neighbor 1293: Europa
Using neighbors: [2010 4234 1824 1293]
Imputed value: Europa

Imputing missing value at row 340, column HomePlanet...
Neighbors: [3655 3314 3309  567]
Neighbor values:
  Neighbor 3655: Earth
  Neighbor 3314: Mars
  Neighbor 3309: Mars
  Neighbor 567: Mars
Using neighbors: [3655 3314 3309  567]
Imputed value: Mars

Imputing missing value at row 450, column HomePlanet...
Neighbors: [3079 1533 1598   44]
Neighbor values:
  Neighbor 3079: Earth
  Neighbor 1533: Earth
  Neighbor 1598: Earth
  Neighbor 44: Earth
Using neighbors: [3079 1533 1598   44]
Imputed value: Earth

Imputing missing value at row 464, column HomePlanet...
Neighbors: [1856 3603 3416 3703]
Neighbor values:
  Neighbor 1856: Earth
  Neighbor 3603: Earth
  Neighbor 3416: Earth
  Neighbor 3703: Earth
Using neighbors: [1856 3603

In [8]:
imputed_df


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.00,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.00,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.00,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.00,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.00,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.00,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,F/1843/P,TRAPPIST-1e,42.00,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,21.00,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,TRAPPIST-1e,42.75,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale
